In [1]:
import os
import json
from pathlib import Path
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, Qdrant
from langchain.document_loaders import TextLoader

In [3]:
json.loads("{}")

{}

In [6]:
a = [{1: "a"}, {2: "b"}]
x = "["+",".join([json.dumps(i) for i in a]) + "]"
print(x)

[{"1": "a"},{"2": "b"}]


In [7]:
y = json.loads(x)
print(y)

[{'1': 'a'}, {'2': 'b'}]


In [49]:
keys_file = Path(".").resolve().parent / "keys.json"
print(keys_file)
with open(keys_file) as f:
    keys = json.load(f)
os.environ["OPENAI_API_KEY"] = keys["OPENAI_API_KEY"]

D:\00Repos\GPT-Swarm\keys.json


In [73]:
text_file_test = Path("D:\\00Repos\\GPT-Swarm\\runs\\run_2023-04-28_14-12-56\\shared_memory.json")
text_file_test = Path("./test_text.txt")
loader = TextLoader(text_file_test)
documents = loader.load()
# # improt json as text
# text_dump = ""
# with open(text_file_test) as f:
#     text_test = json.load(f)
#     for key, val in text_test.items():
#         text_dump += val["content"]

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
texts = text_splitter.split_documents(documents)
len(texts)

Created a chunk of size 1361, which is longer than the specified 1000
Created a chunk of size 1259, which is longer than the specified 1000
Created a chunk of size 1008, which is longer than the specified 1000
Created a chunk of size 1382, which is longer than the specified 1000
Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 1106, which is longer than the specified 1000
Created a chunk of size 1026, which is longer than the specified 1000
Created a chunk of size 1001, which is longer than the specified 1000
Created a chunk of size 1079, which is longer than the specified 1000
Created a chunk of size 1627, which is longer than the specified 1000
Created a chunk of size 1149, which is longer than the specified 1000
Created a chunk of size 1207, which is longer than the specified 1000


75

In [182]:
text_file_test = Path("D:\\00Repos\\GPT-Swarm\\runs\\run_2023-04-28_14-12-56\\shared_memory.json")
# improt json as text
text_dump = ""
with open(text_file_test) as f:
    text_test = json.load(f)
    for key, val in text_test.items():
        text_dump += val["content"]

text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=150, separator=" ")
texts = text_splitter.split_text(text_dump)
len(texts)

93

In [121]:
embeddings = OpenAIEmbeddings()

In [165]:
texts = ["init"]

In [166]:
chroma_db = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    persist_directory="./test_qdrant2",
)

Using embedded DuckDB with persistence: data will be stored in: ./test_qdrant2


In [167]:
chroma_db._collection.count()

1

In [183]:
_ = chroma_db.add_texts(
    texts
)

In [184]:
chroma_db._collection.count()

95

In [193]:
query = "What is the destruction mechanic?"
docs = chroma_db.similarity_search_with_score(query, k=1)
docs = chroma_db.max_marginal_relevance_search(query, k=4)

In [198]:
docs[0][0]

Document(page_content='sessions\n4. High replayability\n5. Minimalistic design\n6. Addictive gameplay\n7. High score-based gameplay\n\nDestruction-Based Game Mechanic Ideas:\n1. Smash and destroy buildings\n2. Explode objects\n3. Break through walls\n4. Crush cars\n5. Demolish structures\n6. Destroy planets\n7. Wreck havoc on cities\n8. Obliterate objects with a hammer\n\nBreaking Down Destruction-Based Game Mechanic Ideas into Smaller Components:\n1. Smash and destroy buildings: Players can use different tools to knock down buildings such as a wrecking ball, bulldozer, or explosives.\n2. Explode objects: Players can set off bombs, grenades, or other explosive devices to destroy objects.\n3. Break through walls: Players can use different tools to break through walls such as a sledgehammer, drill, or pickaxe.\n4. Crush cars: Players can use different tools to crush cars such as a monster truck or a hydraulic press.\n5. Demolish structures: Players can use different tools to demolish str

In [192]:
type(docs[0].page_content)

str

In [155]:
retriever_chroma = chroma_db.as_retriever(search_type="mmr")

In [156]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chains.question_answering import load_qa_chain

In [157]:
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0) # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa_chain = load_qa_chain(model, chain_type="stuff")
qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=retriever_chroma)

In [158]:
questions = [
    "What are the principles of hypercasual game design?",
    "what are the main destruction based machanics?"
]

for question in questions:  
    result = qa.run(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result} \n")

-> **Question**: What are the principles of hypercasual game design? 

**Answer**: The principles of hypercasual game design include the following:
1. Easy to Learn
2. Quick Gameplay
3. Addictive
4. Minimalistic
5. High Replayability
6. Short gameplay sessions
7. Focus on one core mechanic
8. Easy to share and socialize. 

-> **Question**: what are the main destruction based machanics? 

**Answer**: Some main destruction-based game mechanics are:

1. Smash and destroy buildings
2. Explode objects
3. Break through walls
4. Crush cars
5. Demolish structures
6. Destroy planets
7. Wreck havoc on cities
8. Obliterate objects with a hammer

In addition, game mechanics like Chain Reaction, Avalanche, Meteor Strike, Robot Uprising, and Nuclear Fallout also involve destruction-based gameplay objectives. 

